# 🔍 ANÁLISE EXPLORATÓRIA DE DADOS (EDA)

**Objetivo:** Entender os dados e identificar padrões que influenciam a **inadimplência** de clientes.

## 📌 O que vamos descobrir:
- Qualidade dos dados (missing, duplicatas)
- Quem são os inadimplentes?
- Quais variáveis mais impactam o risco de crédito?
- Dados estão prontos para Machine Learning?

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
import warnings
warnings.filterwarnings('ignore')

# Configs de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Bibliotecas carregadas!")

✅ Bibliotecas carregadas!


In [3]:
# Carregando dados do SQLite
DB_PATH = 'data/database.db'

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM clientes", conn)
conn.close()

print(f"📊 Dados carregados: {df.shape[0]:,} clientes × {df.shape[1]} variáveis")
print(f"🎯 Target: inadimplente (0=Adimplente, 1=Inadimplente)")

📊 Dados carregados: 100,000 clientes × 17 variáveis
🎯 Target: inadimplente (0=Adimplente, 1=Inadimplente)


---
# 1️⃣ VISÃO GERAL DOS DADOS
---

In [15]:
print("📋 Amostra dos dados:\n")
display(df.head())

print("📊 Estrutura do dataset:\n")
df.info()

print("🔍 VERIFICAÇÃO DE QUALIDADE")
print("=" * 60)

# Valores nulos
print(f"\n❓ Valores nulos: {df.isnull().sum().sum()}")

# Duplicatas
duplicatas = df.duplicated().sum()
print(f"🔄 Duplicatas: {duplicatas}")

📋 Amostra dos dados:



,id_cliente,idade,genero,estado_civil,escolaridade,estado,renda_anual,valor_patrimonio,possui_imovel_proprio,possui_carro,ocupacao,numero_dependentes,tempo_emprego_atual,score_serasa_externo,utilizacao_limite_cartao,historico_atraso_90_dias,inadimplente
0,1,47,M,Divorciado,Médio,RJ,9059.86,50709.30,1,0,CLT,2,316,698,11.50,0,0
1,2,37,F,Casado,Médio,PR,40713.82,140855.24,1,1,CLT,1,356,389,65.60,2,1
2,3,49,M,Casado,Médio,BA,72448.95,318428.40,1,1,CLT,3,358,428,0.00,0,0
3,4,62,F,Casado,Médio,SP,42668.42,98233.00,1,1,CLT,3,34,560,77.55,1,0
4,5,36,M,Casado,Superior,RS,46197.81,187751.14,0,1,Autônomo,1,290,475,41.52,0,0


📊 Estrutura do dataset:

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   idade                     100000 non-null  int64  
 2   genero                    100000 non-null  str    
 3   estado_civil              100000 non-null  str    
 4   escolaridade              100000 non-null  str    
 5   estado                    100000 non-null  str    
 6   renda_anual               100000 non-null  float64
 7   valor_patrimonio          100000 non-null  float64
 8   possui_imovel_proprio     100000 non-null  int64  
 9   possui_carro              100000 non-null  int64  
 10  ocupacao                  100000 non-null  str    
 11  numero_dependentes        100000 non-null  int64  
 12  tempo_emprego_atual       100000 non-null  int64  
 13  score_serasa_externo      10000

---
# 2️⃣ ANÁLISE DO TARGET (INADIMPLÊNCIA)
---

In [ ]:
# Distribuição da Inadimplência
inadimplencia = df['inadimplente'].value_counts()
percentuais = df['inadimplente'].value_counts(normalize=True) * 100

print("🎯 DISTRIBUIÇÃO DO TARGET")
print("=" * 60)
print(f"\n✅ Adimplentes (0): {inadimplencia[0]:,} ({percentuais[0]:.1f}%)")
print(f"❌ Inadimplentes (1): {inadimplencia[1]:,} ({percentuais[1]:.1f}%)")

# Gráficos
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# G1: Barras
colors = ['#2ecc71', '#e74c3c']
inadimplencia.plot(kind='bar', ax=ax[0], color=colors, edgecolor='black')
ax[0].set_title('Distribuição de Inadimplentes', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Status', fontsize=12)
ax[0].set_ylabel('Quantidade', fontsize=12)
ax[0].set_xticklabels(['Adimplente', 'Inadimplente'], rotation=0)
ax[0].grid(axis='y', alpha=0.3)

# G2: Pizza
ax[1].pie(inadimplencia, labels=['Adimplente', 'Inadimplente'], autopct='%1.1f%%',
          colors=colors, startangle=90, textprops={'fontsize': 12})
ax[1].set_title('Proporção de Inadimplência', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Análise Univariada - CATEGÓRICAS
print("\n" + "="*80)
print("📊 4. ANÁLISE UNIVARIADA - VARIÁVEIS CATEGÓRICAS")
print("="*80)

# Lista de variáveis categóricas
cat_vars = ['genero', 'estado_civil', 'escolaridade', 'estado', 'ocupacao']

# Análise de cada variável
for var in cat_vars:
  print(f"\n📌 VARIÁVEL: {var.upper()}")
  contagem = df[var].value_counts()
  percentual = df[var].value_counts(normalize=True) * 100 

  resumo = pd.DataFrame({
    'Frequência Absoluta': contagem,
    'Percentual (%)': percentual
  })

  resumo['Percentual (%)'] = resumo['Percentual (%)'].map('{:.1f}%'.format)
    
  display(resumo)

# Visualização do gráfico
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

paleta = sns.color_palette("pastel")

for idx, var in enumerate(cat_vars):
  ax = axes[idx]
  contagem = df[var].value_counts()
  contagem.plot(kind='bar', ax=ax, 
                color=paleta[:len(contagem)], 
                edgecolor='black', 
                linewidth=0.8,
                alpha=0.9)
  ax.set_title(f'Distribuição: {var.upper()}', fontweight='bold', fontsize=13, pad=15)
  ax.set_xlabel('')
  ax.set_ylabel('Quantidade', fontsize=10, labelpad=10)
  ax.tick_params(axis='x', rotation=45, labelsize=9)
  ax.grid(axis='y', alpha=0.2, linestyle='--')
  sns.despine(ax=ax)
  offset = max(contagem) * 0.02
  
  for i, val in enumerate(contagem):
    ax.text(i, val + offset, f'{val}', ha='center', va='bottom', 
            fontsize=9, fontweight='bold', color='#444444')
    
for i in range(len(cat_vars), len(axes)):
    axes[i].axis('off')

plt.tight_layout(pad=4.0) 
plt.show()

In [ ]:
# Análise Univariada - NUMÉRICAS
print("\n" + "="*80)
print("📊 5. ANÁLISE UNIVARIADA - VARIÁVEIS NUMÉRICAS")
print("="*80)

num_vars = [
    'idade', 'renda_anual', 'valor_patrimonio', 'numero_dependentes',
    'tempo_emprego_atual', 'score_serasa_externo', 'utilizacao_limite_cartao',
    'historico_atraso_90_dias'
]

for var in num_vars:
    print(f"\n📌 VARIÁVEL: {var.upper()}")

    print(f"\n📊 Estatísticas:")
    print(f"Média: {df[var].mean():,.2f}")
    print(f"Mediana: {df[var].median():,.2f}")
    print(f"Desvio Padrão: {df[var].std():,.2f}")
    print(f"Mínimo: {df[var].min():,.2f}")
    print(f"Máximo: {df[var].max():,.2f}")
    print(f"Q1 (25%): {df[var].quantile(0.25):,.2f}")
    print(f"Q3 (75%): {df[var].quantile(0.75):,.2f}")

    # Detecta outliers (método IQR)
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1

    outliers = (
        (df[var] < (Q1 - 1.5 * IQR)) |
        (df[var] > (Q3 + 1.5 * IQR))
    ).sum()

    print(f"\n🔍 Outliers detectados: {outliers:,} ({outliers / len(df) * 100:.2f}%)")

# =======================
# VISUALIZAÇÕES
# =======================

fig, axes = plt.subplots(len(num_vars), 2, figsize=(16, 4 * len(num_vars)))

for idx, var in enumerate(num_vars):
    # Histograma
    axes[idx, 0].hist(
        df[var].dropna(),
        bins=50,
        color='steelblue',
        edgecolor='black',
        alpha=0.7
    )

    axes[idx, 0].axvline(
        df[var].mean(),
        color='red',
        linestyle='--',
        linewidth=2,
        label=f'Média: {df[var].mean():.0f}'
    )

    axes[idx, 0].axvline(
        df[var].median(),
        color='green',
        linestyle='--',
        linewidth=2,
        label=f'Mediana: {df[var].median():.0f}'
    )

    axes[idx, 0].set_title(f'Histograma: {var.upper()}', fontweight='bold')
    axes[idx, 0].set_xlabel(var)
    axes[idx, 0].set_ylabel('Frequência')
    axes[idx, 0].legend()
    axes[idx, 0].grid(alpha=0.3, linestyle='--')

    # Boxplot
    axes[idx, 1].boxplot(
        df[var].dropna(),
        vert=False,
        patch_artist=True,
        boxprops=dict(facecolor='lightblue', alpha=0.7),
        medianprops=dict(color='red', linewidth=2),
        whiskerprops=dict(color='black', linewidth=1.5),
        capprops=dict(color='black', linewidth=1.5)
    )

    axes[idx, 1].set_title(f'Boxplot: {var.upper()}', fontweight='bold')
    axes[idx, 1].set_xlabel(var)
    axes[idx, 1].grid(alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()


In [ ]:
# Análise Bivariada - TARGET vs VAR CATEGÓRICAS
print("\n" + "="*80)
print("🎯 6. ANÁLISE BIVARIADA - INADIMPLÊNCIA vs CATEGÓRICAS")
print("="*80)

# Para cada var categórica, ver a taxa de inadimplência
for var in cat_vars:
  print(f"\n📌 TAXA DE INADIMPLÊNCIA POR: {var.upper()}")

  taxa_por_categoria = (
      df.groupby(var)['inadimplente']
        .agg(['sum', 'count', 'mean'])
  )

  taxa_por_categoria.columns = ['Inadimplentes', 'Total', 'Taxa']
  taxa_por_categoria['Taxa'] = taxa_por_categoria['Taxa'] * 100
  taxa_por_categoria = taxa_por_categoria.sort_values('Taxa', ascending=False)

  print(f"\n{taxa_por_categoria.to_string()}")

  # Identifica categoria de maior e menor risco
  maior_risco = taxa_por_categoria['Taxa'].idxmax()
  menor_risco = taxa_por_categoria['Taxa'].idxmin()

  print(f"\n💡 Insights:")
  print(f"• Maior risco: {maior_risco} ({taxa_por_categoria.loc[maior_risco, 'Taxa']:.1f}%)")
  print(f"• Menor risco: {menor_risco} ({taxa_por_categoria.loc[menor_risco, 'Taxa']:.1f}%)")


# Visualizações
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, var in enumerate(cat_vars):
  taxa = (
      df.groupby(var)['inadimplente']
        .mean()
        .sort_values(ascending=False) * 100
  )

  taxa.plot(
      kind='barh',
      ax=axes[idx],
      color='coral',
      edgecolor='black',
      alpha=0.8
  )

  axes[idx].set_title(f"Taxa de Inadimplência por {var.upper()}", fontweight='bold')
  axes[idx].set_xlabel("Taxa de Inadimplência (%)")
  axes[idx].grid(axis='x', alpha=0.3, linestyle='--')

  # Linha de referência da taxa geral
  taxa_geral = df['inadimplente'].mean() * 100
  axes[idx].axvline(
      taxa_geral,
      color='red',
      linestyle='--',
      linewidth=2,
      label=f"Taxa Geral: {taxa_geral:.1f}%"
  )

  axes[idx].legend()
  
# Remove gráficos vazios, se houver
for j in range(len(cat_vars), len(axes)):
  axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Análise Bivariada - TARGET vs VAR NUMÉRICAS
print("\n" + "="*80)
print("🎯 7. ANÁLISE BIVARIADA - INADIMPLÊNCIA vs NUMÉRICAS")
print("="*80)

# Comparação de médias entre adimplentes e inadimplentes
print("\n📊 Comparação de Médias (Adimplentes vs Inadimplentes):")

comparacao = df.groupby('inadimplente')[num_vars].mean().T
comparacao.columns = ['Adimplente', 'Inadimplente']
comparacao['Diferença'] = comparacao['Inadimplente'] - comparacao['Adimplente']
comparacao['% Diferença'] = (comparacao['Diferença'] / comparacao['Adimplente']) * 100

print(comparacao.to_string())

# Visualização - Boxplots comparativos
fig, axes = plt.subplots(4, 2, figsize=(16, 20))
axes = axes.flatten()

for idx, var in enumerate(num_vars):
  adim = df[df['inadimplente'] == 0][var]
  inadim = df[df['inadimplente'] == 1][var]
    
  box_data = [adim, inadim]
  bp = axes[idx].boxplot(box_data, labels=['Adimplente', 'Inadimplente'],
                          patch_artist=True, widths=0.6)
    
  bp['boxes'][0].set_facecolor('lightgreen')
  bp['boxes'][1].set_facecolor('lightcoral')
    
  axes[idx].set_title(f'{var.upper()}', fontweight='bold', fontsize=12)
  axes[idx].set_ylabel(var)
  axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
    
  # Adiciona médias
  axes[idx].plot([1, 2], [adim.mean(), inadim.mean()], 
                  'ro-', linewidth=2, markersize=10, label='Média')
  axes[idx].legend()

plt.tight_layout()
plt.show()

In [ ]:
# MATRIZ DE CORRELAÇÃO
print("\n" + "="*80)
print("📊 8. MATRIZ DE CORRELAÇÃO")
print("="*80)

# Seleciona apenas variáveis numéricas
df_numeric = df.select_dtypes(include=[np.number])

# Calcula correlação
correlacao = df_numeric.corr()

# Mostra correlações com o target
print("\n🎯 Correlação com INADIMPLENTE (ordenado):")
print("="*80)
corr_target = correlacao['inadimplente'].sort_values(ascending=False)
for var, valor in corr_target.items():
    if var != 'inadimplente':
        simbolo = "⬆️" if valor > 0 else "⬇️"
        forca = "FORTE" if abs(valor) > 0.5 else "MODERADA" if abs(valor) > 0.3 else "FRACA"
        print(f"   {var:<30} {simbolo} {valor:>6.3f} ({forca})")

# Heatmap de correlação
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(correlacao, dtype=bool))  # Máscara para triângulo superior

sns.heatmap(correlacao, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True,
            linewidths=1, cbar_kws={"shrink": 0.8},
            vmin=-1, vmax=1)

plt.title('Matriz de Correlação - Variáveis Numéricas', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 Interpretação:")
print("   • Correlação perto de +1: relação positiva forte")
print("   • Correlação perto de -1: relação negativa forte")
print("   • Correlação perto de 0: sem relação linear")